In [4]:
import pandas as pd 
import seaborn as sns
from statistics import mean, variance
from scipy import stats 
import seaborn as sns 
import matplotlib.pyplot as plt
from feature_engine.selection import DropCorrelatedFeatures
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np 
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.model_selection import cross_val_score, GridSearchCV , KFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.decomposition import PCA
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor 
import plotly.graph_objects as go
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import BayesianRidge
from numpy import arange
from xgboost.sklearn import XGBRegressor
import lightgbm as lgb

###### IMPORT DATA 

In [3]:
df = pd.read_excel('trainDataset.xls')
df = df.drop('ID', axis=1)
df = df.drop('pCR (outcome)', axis = 1)

###### DATA INSPECTION

In [31]:
df.head()
df.describe()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 118 entries, RelapseFreeSurvival (outcome) to original_ngtdm_Strength
dtypes: float64(108), int64(10)
memory usage: 368.9 KB


## DATA PREPROCESSING

imputing missing values with median 

In [5]:
imp = SimpleImputer(missing_values=999, strategy='median')
# Fitting the data to the imputer object
imputer = imp.fit(df)
imputed_df = imputer.transform(df)
imputed_df = pd.DataFrame(imputed_df, columns = df.columns)

In [7]:
 scaled_x, scaled_test_x = normalisation(x_train, x_test)

NameError: name 'x_train' is not defined

In [ ]:
def de_correlate_df(df):
    X_aux = df.copy()
    for col in df.columns:
        X_aux[col] = df[col].sample(len(df)).values
    return X_aux

pca = PCA()
pca.fit(scaled_x)
original_variance = pca.explained_variance_ratio_

N_permutations = 1000
variance = np.zeros((N_permutations, len(scaled_x.columns)))

for i in range(N_permutations):
    X_aux = de_correlate_df(scaled_x)
    pca.fit(X_aux)
    variance[i, :] = pca.explained_variance_ratio_
    
p_val = np.sum(variance > original_variance, axis=0) / N_permutations
fig = go.Figure()
fig.add_trace(go.Scatter(x=[f'PC{i}' for i in range(len(df.columns))], y=p_val, name='p-value on significance'))
fig.update_layout(title="PCA Permutation Test p-values")

###### feature normalisation

In [6]:
def normalisation(x_train, x_test):
    scaler = MinMaxScaler()
    scaled_x = scaler.fit_transform(x_train)
    scaled_x = pd.DataFrame(scaled_x)
    scaled_test_x = scaler.fit_transform(x_test)
    return scaled_x , scaled_test_x

###### Parameter tuning

### XGBoost

###### param tuning

In [20]:
def XGB_param_tuning(cv_inner):
    xgb1 = XGBRegressor()
    parameters = {
                  'learning_rate': [.03, 0.05, .07, 0.1],
                  'max_depth': [5, 6, 7],
                  'subsample': [0.5, 0.7,1.0],
                  'colsample_bytree': [0.7],
                  'n_estimators': [100, 250, 500]
    }
      
    xgb_grid = GridSearchCV(xgb1,
                            parameters,
                            cv = cv_inner, scoring='neg_mean_absolute_error',
                            n_jobs = -1,
                            verbose=True)
    xgb_grid.fit(x_PCA,y_train)

    return xgb_grid

### lightbgm

In [19]:
# grid search  hyperparameter tuning
def lgm_param(cv_inner):
    parameters = {
        'boosting': ['gbdt', 'dart','goss' ],
        'lambda_l1':[0.01, 0.1, 0.5],
       'num_leaves':[ 7, 15, 31  ],
       'max_depth' :[ 10,15,25],
       'min_data_in_leaf':[15,25]
    }
    
    lgb_model = lgb.LGBMRegressor()

    gsearch_lgb = GridSearchCV(lgb_model,scoring = 'neg_mean_absolute_error', param_grid = parameters, n_jobs=-1,verbose=10)
    gsearch_lgb.fit(x_PCA,y_train)
    return gsearch_lgb

### SVM

In [23]:
#Parameter tuning for support vector machine
def svr_param(cv_inner):
    SVR_param_grid = {'C':[0.01, 0.1, 1,10],'kernel':['linear','rbf','poly'],'gamma':[0.001,0.01,0.1,1,10], 'epsilon':[0.01,0.1,1,10]}

    #Find the best parameters
    svr_grid = GridSearchCV(SVR(), SVR_param_grid, cv=cv_inner, scoring='neg_mean_absolute_error')
    svr_grid.fit(x_PCA, np.ravel(y_train))
    return svr_grid

### Decision Tree 

In [8]:
#Parameter tuning for decision tree
def dt_param(cv_inner):
    param = {
        'splitter': ['best','random'],
        'max_depth': [1,3,5,7,9,11,12],
        'min_samples_leaf': [1,2,3,4,5,6,7,8,9,10],
        'min_weight_fraction_leaf':[0.1,0.2,0.3,0.4,0.5],
        'max_features':["auto","log2","sqrt",None],
        'max_leaf_nodes':[None,10,20,30,40,50,60,70,80,90]
            }

    #define grid search
    tree_grid = GridSearchCV(DecisionTreeRegressor(), param_grid = param, cv = cv_inner,n_jobs = -1, verbose = 3, scoring = 'neg_mean_absolute_error' )
    tree_grid.fit(x_PCA,y_train)
    return tree_grid

### Bayesian Ridge Regression


In [17]:
 def br_param(cv_inner): 
    model = BayesianRidge()
    # define model evaluation method
    cv =KFold(n_splits = 7)
    # define grid
    grid = {'alpha_init':[0.3,0.5,0.8,1.0] , 'lambda_init':[1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-9]}

    # define search
    search = GridSearchCV(model, grid, scoring='neg_mean_absolute_error', cv=cv_inner, n_jobs=-1)
    # perform the search
    results = search.fit(x_PCA, y_train)
    return results

### NESTED CROSS VALIDATION

###### XGBOOST

In [7]:
rfs = np.array(imputed_df['RelapseFreeSurvival (outcome)'])
features = np.array(imputed_df.drop('RelapseFreeSurvival (outcome)', axis =1 ))

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
outer_results = list()
for train_ix, test_ix in cv_outer.split(features):
    # split data
    x_train, x_test = features[train_ix, :], features[test_ix, :]
    y_train, y_test = rfs[train_ix], rfs[test_ix]
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    
    x_train = pd.DataFrame(x_train)
    
    scaled_x, scaled_test_x = normalisation(x_train, x_test)
    pca = PCA(n_components=11)
    x_PCA = pca.fit_transform(scaled_x)
    x_test = pca.fit_transform(scaled_test_x)

    search = XGB_param_tuning(cv_inner)
     # get the best performing model fit on the whole training set
    best_model = search.best_estimator_
    pred = best_model.predict(x_test)
    mae = mean_absolute_error(y_test,pred)
    outer_results.append(mae)
    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae, search.best_score_, search.best_params_))
print('Mean Absolute Error after parameter tuning: {:.2f}'.format(mean(outer_results)))
print('variance after parameter tuning: {:.2f}'.format(variance(outer_results)))

NameError: name 'normalisation' is not defined

###### LIGHT GBM 

In [46]:
rfs = np.array(imputed_df['RelapseFreeSurvival (outcome)'])
features = np.array(imputed_df.drop('RelapseFreeSurvival (outcome)', axis =1 ))

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
outer_results = list()
for train_ix, test_ix in cv_outer.split(features):
    # split data
    x_train, x_test = features[train_ix, :], features[test_ix, :]
    y_train, y_test = rfs[train_ix], rfs[test_ix]
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    
    x_train = pd.DataFrame(x_train)
    scaled_x, scaled_test_x = normalisation(x_train, x_test)
    pca = PCA(n_components=11)
    x_PCA = pca.fit_transform(scaled_x)
    x_test = pca.fit_transform(scaled_test_x)

    search = lgm_param(cv_inner)
     # get the best performing model fit on the whole training set
    best_model = search.best_estimator_
    pred = best_model.predict(x_test)
    mae = mean_absolute_error(y_test,pred)
    outer_results.append(mae)
    print('-------------------------------------------------')
    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae, search.best_score_, search.best_params_))
print('Mean Absolute Error after parameter tuning: {:.2f}'.format(mean(outer_results)))
print('variance after parameter tuning: {:.2f}'.format(variance(outer_results)))


Fitting 5 folds for each of 162 candidates, totalling 810 fits
[LightGBM] [Warning] boosting is set=goss, boosting_type=gbdt will be ignored. Current value: boosting=goss
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] lambda_l1 is set=0.01, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.01
-------------------------------------------------
>mae=21.364, est=-22.199, cfg={'boosting': 'goss', 'lambda_l1': 0.01, 'max_depth': 10, 'min_data_in_leaf': 25, 'num_leaves': 7}
Fitting 5 folds for each of 162 candidates, totalling 810 fits
[LightGBM] [Warning] boosting is set=dart, boosting_type=gbdt will be ignored. Current value: boosting=dart
[LightGBM] [Warning] min_data_in_leaf is set=15, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=15
[LightGBM] [Warning] lambda_l1 is set=0.5, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.5
----------------------------

###### BAYESIAN RIDGE REGRESSION 

In [47]:
rfs = np.array(imputed_df['RelapseFreeSurvival (outcome)'])
features = np.array(imputed_df.drop('RelapseFreeSurvival (outcome)', axis =1 ))

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
outer_results = list()
for train_ix, test_ix in cv_outer.split(features):
    # split data
    x_train, x_test = features[train_ix, :], features[test_ix, :]
    y_train, y_test = rfs[train_ix], rfs[test_ix]
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    
    x_train = pd.DataFrame(x_train)
    
    scaled_x, scaled_test_x = normalisation(x_train, x_test)
    pca = PCA(n_components=11)
    x_PCA = pca.fit_transform(scaled_x)
    x_test = pca.fit_transform(scaled_test_x)

    search = br_param(cv_inner)
     # get the best performing model fit on the whole training set
    best_model = search.best_estimator_
    pred = best_model.predict(x_test)
    mae = mean_absolute_error(y_test,pred)
    outer_results.append(mae)
    print('-------------------------------------------------')
    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae, search.best_score_, search.best_params_))
print('Mean Absolute Error after parameter tuning: {:.2f}'.format(mean(outer_results)))
print('variance after parameter tuning: {:.2f}'.format(variance(outer_results)))

-------------------------------------------------
>mae=20.671, est=-20.904, cfg={'alpha_init': 0.5, 'lambda_init': 1e-09}
-------------------------------------------------
>mae=21.239, est=-20.920, cfg={'alpha_init': 0.3, 'lambda_init': 0.1}
-------------------------------------------------
>mae=19.198, est=-20.849, cfg={'alpha_init': 0.3, 'lambda_init': 1e-09}
-------------------------------------------------
>mae=25.343, est=-20.293, cfg={'alpha_init': 0.3, 'lambda_init': 1e-09}
-------------------------------------------------
>mae=16.305, est=-21.347, cfg={'alpha_init': 0.3, 'lambda_init': 0.1}
-------------------------------------------------
>mae=24.094, est=-20.352, cfg={'alpha_init': 0.3, 'lambda_init': 1e-09}
-------------------------------------------------
>mae=24.909, est=-20.523, cfg={'alpha_init': 0.3, 'lambda_init': 0.1}
-------------------------------------------------
>mae=17.773, est=-21.039, cfg={'alpha_init': 0.3, 'lambda_init': 1e-09}
------------------------------

###### DECISION TREE 

In [9]:
rfs = np.array(imputed_df['RelapseFreeSurvival (outcome)'])
features = np.array(imputed_df.drop('RelapseFreeSurvival (outcome)', axis =1 ))

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
outer_results = list()
for train_ix, test_ix in cv_outer.split(features):
    # split data
    x_train, x_test = features[train_ix, :], features[test_ix, :]
    y_train, y_test = rfs[train_ix], rfs[test_ix]
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    
    x_train = pd.DataFrame(x_train)
    scaled_x, scaled_test_x = normalisation(x_train, x_test)
    pca = PCA(n_components=11)
    x_PCA = pca.fit_transform(scaled_x)
    x_test = pca.fit_transform(scaled_test_x)

    search = dt_param(cv_inner)
     # get the best performing model fit on the whole training set
    best_model = search.best_estimator_
    pred = best_model.predict(x_test)
    mae = mean_absolute_error(y_test,pred)
    outer_results.append(mae)
    print('-------------------------------------------------')
    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae, search.best_score_, search.best_params_))
print('Mean Absolute Error after parameter tuning: {:.2f}'.format(mean(outer_results)))
print('variance after parameter tuning: {:.2f}'.format(variance(outer_results)))

Fitting 5 folds for each of 28000 candidates, totalling 140000 fits
-------------------------------------------------
>mae=21.455, est=-20.042, cfg={'max_depth': 1, 'max_features': 'sqrt', 'max_leaf_nodes': 30, 'min_samples_leaf': 3, 'min_weight_fraction_leaf': 0.2, 'splitter': 'best'}
Fitting 5 folds for each of 28000 candidates, totalling 140000 fits
-------------------------------------------------
>mae=20.717, est=-20.291, cfg={'max_depth': 12, 'max_features': 'sqrt', 'max_leaf_nodes': 50, 'min_samples_leaf': 9, 'min_weight_fraction_leaf': 0.3, 'splitter': 'best'}
Fitting 5 folds for each of 28000 candidates, totalling 140000 fits
-------------------------------------------------
>mae=20.169, est=-20.383, cfg={'max_depth': 7, 'max_features': None, 'max_leaf_nodes': 20, 'min_samples_leaf': 10, 'min_weight_fraction_leaf': 0.1, 'splitter': 'random'}
Fitting 5 folds for each of 28000 candidates, totalling 140000 fits
-------------------------------------------------
>mae=25.039, est=-1

###### SUPPORT VECTOR MACHINE

In [ ]:
rfs = np.array(imputed_df['RelapseFreeSurvival (outcome)'])
features = np.array(imputed_df.drop('RelapseFreeSurvival (outcome)', axis =1 ))

cv_outer = KFold(n_splits=10, shuffle=True, random_state=1)
outer_results = list()
for train_ix, test_ix in cv_outer.split(features):
    # split data
    x_train, x_test = features[train_ix, :], features[test_ix, :]
    y_train, y_test = rfs[train_ix], rfs[test_ix]
    cv_inner = KFold(n_splits=5, shuffle=True, random_state=1)
    
    x_train = pd.DataFrame(x_train)
    scaled_x, scaled_test_x = normalisation(x_train, x_test)
    pca = PCA(n_components=11)
    x_PCA = pca.fit_transform(scaled_x)
    x_test = pca.fit_transform(scaled_test_x)

    search = svr_param(cv_inner)
     # get the best performing model fit on the whole training set
    best_model = search.best_estimator_
    pred = best_model.predict(x_test)
    mae = mean_absolute_error(y_test,pred)
    outer_results.append(mae)
    print('-------------------------------------------------')
    print('>mae=%.3f, est=%.3f, cfg=%s' % (mae, search.best_score_, search.best_params_))
print('Mean Absolute Error after parameter tuning: {:.2f}'.format(mean(outer_results)))
print('variance after parameter tuning: {:.2f}'.format(variance(outer_results)))